import pandas as pd 
from collections import defaultdict
import math
import copy
import json
import ast
data= pd.read_csv('input_p.csv')

In [1]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
df_demand = {}
random.seed(12) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    df_demand['D'+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v
    
df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0
    
np.random.seed(12) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,40, size=(sum,20))).astype(float)

random.seed(12) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)
    
age = {}
for i in range(len(age_b)):
    age[i] = age_b[i] 
    
kpl = []
co2 = []
random.seed(12) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)
        
fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])
    
random.seed(12) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n
    
df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
df_tdkom

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,239.445034,830.597653,357.466468,599.909783,77.595894,993.975858,971.027782,136.291951,1047.717146,228.948085,372.432960,697.710601,1007.812703,931.666487,72.932055,625.647903,613.898063,585.679010,829.353012,256.574290
1,817.978745,114.921987,224.916571,180.592190,359.416494,723.829626,527.064599,884.186333,373.953857,794.221994,762.848941,408.861802,389.838700,1020.694325,668.322913,1017.422563,799.982969,893.075871,455.242988,528.633136
2,666.827995,1529.130985,356.302799,1456.688428,670.361521,695.447678,746.518425,645.001340,770.731309,148.707164,217.012029,1152.449234,988.421462,130.217773,507.043546,430.402687,150.064134,860.000655,669.415102,171.709939
3,855.009923,1172.777657,251.925597,444.090498,924.330995,743.633073,757.051072,723.619287,1296.255652,905.920132,900.522371,692.707434,715.213812,1275.321809,353.444407,1373.577881,167.955233,902.599553,112.232566,989.622640
4,855.962861,1135.959028,1231.678727,608.610581,871.973241,1196.269423,824.322010,493.935395,484.205882,446.031503,1245.256324,586.811018,1211.541583,1118.740914,902.371298,114.487608,1151.318956,1213.098720,76.073904,873.248983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,286.904467,125.967012,693.999723,923.778202,404.124363,849.525054,276.080683,294.512572,819.700913,459.379119,293.133550,603.948524,702.766107,731.784379,128.434159,221.807506,308.615720,710.574133,87.904576,513.507717
2068,740.937170,1127.625990,301.375523,1065.132739,123.374037,823.598868,561.954893,507.654456,589.027377,931.530273,325.731534,585.207549,245.233763,1183.856942,975.983529,339.717123,817.936749,918.436543,715.977415,391.137775
2069,1192.079391,206.195060,574.141054,571.511753,592.584881,885.202795,403.753432,178.140348,1263.500324,783.352765,612.987716,1033.326001,1129.952700,742.450410,512.204987,1002.201227,936.112342,1284.179290,543.137194,986.440559
2070,1110.051854,473.466453,278.063601,528.170988,179.915577,740.410165,429.108582,655.192171,347.818444,1139.760265,907.494909,381.498643,113.268569,909.479681,203.897713,198.298976,576.230632,412.949745,323.912566,491.806575


In [2]:
df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]

In [3]:
df_tdkom

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20
0,239.445034,830.597653,357.466468,599.909783,77.595894,993.975858,971.027782,136.291951,1047.717146,228.948085,372.432960,697.710601,1007.812703,931.666487,72.932055,625.647903,613.898063,585.679010,829.353012,256.574290
1,817.978745,114.921987,224.916571,180.592190,359.416494,723.829626,527.064599,884.186333,373.953857,794.221994,762.848941,408.861802,389.838700,1020.694325,668.322913,1017.422563,799.982969,893.075871,455.242988,528.633136
2,666.827995,1529.130985,356.302799,1456.688428,670.361521,695.447678,746.518425,645.001340,770.731309,148.707164,217.012029,1152.449234,988.421462,130.217773,507.043546,430.402687,150.064134,860.000655,669.415102,171.709939
3,855.009923,1172.777657,251.925597,444.090498,924.330995,743.633073,757.051072,723.619287,1296.255652,905.920132,900.522371,692.707434,715.213812,1275.321809,353.444407,1373.577881,167.955233,902.599553,112.232566,989.622640
4,855.962861,1135.959028,1231.678727,608.610581,871.973241,1196.269423,824.322010,493.935395,484.205882,446.031503,1245.256324,586.811018,1211.541583,1118.740914,902.371298,114.487608,1151.318956,1213.098720,76.073904,873.248983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,286.904467,125.967012,693.999723,923.778202,404.124363,849.525054,276.080683,294.512572,819.700913,459.379119,293.133550,603.948524,702.766107,731.784379,128.434159,221.807506,308.615720,710.574133,87.904576,513.507717
2068,740.937170,1127.625990,301.375523,1065.132739,123.374037,823.598868,561.954893,507.654456,589.027377,931.530273,325.731534,585.207549,245.233763,1183.856942,975.983529,339.717123,817.936749,918.436543,715.977415,391.137775
2069,1192.079391,206.195060,574.141054,571.511753,592.584881,885.202795,403.753432,178.140348,1263.500324,783.352765,612.987716,1033.326001,1129.952700,742.450410,512.204987,1002.201227,936.112342,1284.179290,543.137194,986.440559
2070,1110.051854,473.466453,278.063601,528.170988,179.915577,740.410165,429.108582,655.192171,347.818444,1139.760265,907.494909,381.498643,113.268569,909.479681,203.897713,198.298976,576.230632,412.949745,323.912566,491.806575


In [4]:
dk = df_tdkom.transpose()

In [5]:
dk

,0,1,2,3,4,5,6,7,8,9,...,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
D1,239.445034,817.978745,666.827995,855.009923,855.962861,462.668799,250.003548,1111.521113,621.972533,1080.215926,...,305.324524,1254.374472,1022.879936,1196.473687,765.914701,286.904467,740.937170,1192.079391,1110.051854,557.916369
D2,830.597653,114.921987,1529.130985,1172.777657,1135.959028,677.611121,441.038206,1020.156899,635.266150,1047.081095,...,716.127826,715.966522,428.943723,200.183654,490.339412,125.967012,1127.625990,206.195060,473.466453,367.950442
D3,357.466468,224.916571,356.302799,251.925597,1231.678727,692.784867,772.918010,1062.063023,106.089907,151.925274,...,865.575697,962.396872,156.276442,611.249068,326.306404,693.999723,301.375523,574.141054,278.063601,227.588376
D4,599.909783,180.592190,1456.688428,444.090498,608.610581,655.657736,286.966110,250.200736,571.042083,905.413935,...,350.206031,642.910708,832.656541,91.148644,441.426874,923.778202,1065.132739,571.511753,528.170988,493.987688
D5,77.595894,359.416494,670.361521,924.330995,871.973241,368.188134,155.895955,398.984550,742.367720,178.231093,...,768.236624,710.765976,1404.715559,262.955278,209.869994,404.124363,123.374037,592.584881,179.915577,451.439404
D6,993.975858,723.829626,695.447678,743.633073,1196.269423,149.416237,882.394674,337.830416,911.300559,1057.529069,...,791.701177,1131.693972,1229.645117,1089.714221,824.541004,849.525054,823.598868,885.202795,740.410165,244.063695
D7,971.027782,527.064599,746.518425,757.051072,824.322010,858.828802,183.633754,596.320898,690.140595,710.519863,...,188.016604,923.271323,1045.957022,1288.183655,710.055345,276.080683,561.954893,403.753432,429.108582,103.085183
D8,136.291951,884.186333,645.001340,723.619287,493.935395,993.857667,623.272679,679.176170,203.318897,1168.374309,...,693.184240,1213.143596,824.255808,731.402428,812.200394,294.512572,507.654456,178.140348,655.192171,656.160390
D9,1047.717146,373.953857,770.731309,1296.255652,484.205882,287.206489,345.687528,526.723187,311.473473,559.853459,...,287.156768,1307.236797,853.816717,722.751760,500.594405,819.700913,589.027377,1263.500324,347.818444,123.091719
D10,228.948085,794.221994,148.707164,905.920132,446.031503,298.324916,631.393291,878.629103,457.872631,772.087143,...,802.799841,972.191877,400.982885,125.694100,764.034852,459.379119,931.530273,783.352765,1139.760265,289.936189


In [6]:
costs_t = dk.to_dict()
costs_t

{0: {'D1': 239.44503375328873,
  'D2': 830.5976532854733,
  'D3': 357.46646817034,
  'D4': 599.9097826705025,
  'D5': 77.59589368212282,
  'D6': 993.975857912804,
  'D7': 971.0277823917804,
  'D8': 136.29195140021335,
  'D9': 1047.7171455062944,
  'D10': 228.94808488290505,
  'D11': 372.43296005015236,
  'D12': 697.7106011959303,
  'D13': 1007.8127029664902,
  'D14': 931.6664868726566,
  'D15': 72.93205467289387,
  'D16': 625.6479025462759,
  'D17': 613.8980633564664,
  'D18': 585.6790103116599,
  'D19': 829.3530120918232,
  'D20': 256.5742904365643},
 1: {'D1': 817.9787447859499,
  'D2': 114.92198675569861,
  'D3': 224.9165705814622,
  'D4': 180.59219007674284,
  'D5': 359.416493780316,
  'D6': 723.8296257763311,
  'D7': 527.0645992042038,
  'D8': 884.186332564618,
  'D9': 373.9538574687085,
  'D10': 794.2219939239175,
  'D11': 762.8489405834597,
  'D12': 408.8618021688384,
  'D13': 389.8386999155583,
  'D14': 1020.6943253389177,
  'D15': 668.3229125554659,
  'D16': 1017.4225634804022

costs=ast.literal_eval(data['Costs'].iloc[0])

costs

In [7]:
df_demand

{'D1': 110,
 'D2': 84,
 'D3': 134,
 'D4': 117,
 'D5': 135,
 'D6': 94,
 'D7': 68,
 'D8': 98,
 'D9': 51,
 'D10': 97,
 'D11': 111,
 'D12': 85,
 'D13': 132,
 'D14': 108,
 'D15': 138,
 'D16': 126,
 'D17': 79,
 'D18': 121,
 'D19': 50,
 'D20': 134}

output=[]
costs_list=[]
for instance in range(640):
	print(instance+1)
	costs=ast.literal_eval(data['Costs'].iloc[instance])
	demand=ast.literal_eval(data['Demand'].iloc[instance])
	supply=ast.literal_eval(data['Supply'].iloc[instance])
	cols = sorted(demand.keys())

In [8]:
colsn = sorted(df_demand.keys())

In [9]:
    
	# print(supply['S1'])
	# break

	costs1=copy.deepcopy(costs_t)
	costs2=copy.deepcopy(costs_t)
	costs3=copy.deepcopy(costs_t)
	for i in df_supply:
	    mi=min(costs_t[i].values())
	    # print(costs[i])
	    # print(mi)
	    for j in costs2[i]:
	        costs2[i][j]-=mi
	# print(costs2)
	for i in df_demand :
	    mi=10000
	    for j in df_supply:
	        if costs_t[j][i]<mi :
	            mi=costs_t[j][i]
	    for j in df_supply:
	        costs3[j][i]=costs3[j][i]-mi 
	# print(costs3)

	for i in df_demand:
	    for j in df_supply:
	        costs_t[j][i]= costs2[j][i]+costs3[j][i]
	# print(costs_t)

	            

	res = dict((k, defaultdict(int)) for k in costs_t)
	g = {}
	for x in df_supply:
		# print(x)
		# print(costs_t[x])
		g[x] = sorted(costs_t[x].keys(), key=lambda g: costs_t[x][g])
	for x in df_demand:
	    g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])

	 
	while g:
	    d = {}
	    # print(df_supply,dic)
	    for x in df_demand:
	        d[x] = (costs_t[g[x][1]][x] - costs_t[g[x][0]][x]) if len(g[x]) > 1 else (costs_t[g[x][0]][x])
	    s = {}
	    for x in df_supply:
	        s[x] = (costs_t[x][g[x][1]] - costs_t[x][g[x][0]])  if len(g[x]) > 1 else costs_t[x][g[x][0]]
	    f = max(d, key=lambda n: d[n])
	    t = max(s, key=lambda n: s[n])
	    t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)
	    v = min(df_supply[f], df_demand[t])
	    # print(f,t)
	    # print(v)
	    res[f][t] += v
	    df_demand[t] -= v
	    
	    if df_demand[t] == 0:
	        for k, n in df_supply.items():
	            if n != 0:
	                g[k].remove(t)
	        del g[t]
	        del df_demand[t]
	    df_supply[f] -= v
	    if df_supply[f] == 0:
	        for k, n in df_demand.items():
	            if n != 0:
	                g[k].remove(f)
	        del g[f]
	        del df_supply[f]
	 
	# print("G",g)
	cost = 0
	# cols = sorted(df_demand.keys())
	# print(costs)
	for g in sorted(costs1):
	    # print (g, " ",)
	    # print("S")
	    for n in colsn:
	        y = res[g][n]
	        # print("YESS",y)
	        if y != 0:
	            pass
	            # print (y,)
	        cost += y * costs1[g][n]
	        # print ("  ",)
	    # print(" ")
	
	#costs_list.append(cost)
	

In [10]:
cost

289131.3038285154

    
	# print(supply['S1'])
	# break

	costs1=copy.deepcopy(costs)

	costs2=copy.deepcopy(costs)
	costs3=copy.deepcopy(costs)
	for i in supply:
	    mi=min(costs[i].values())
	    # print(costs[i])
	    # print(mi)
	    for j in costs2[i]:
	        costs2[i][j]-=mi
	# print(costs2)
	for i in demand :
	    mi=10000
	    for j in supply:
	        if costs[j][i]<mi :
	            mi=costs[j][i]
	    for j in supply:
	        costs3[j][i]=costs3[j][i]-mi 
	# print(costs3)

	for i in demand:
	    for j in supply:
	        costs[j][i]= costs2[j][i]+costs3[j][i]
	# print(costs)

	            

	res = dict((k, defaultdict(int)) for k in costs)
	g = {}
	for x in supply:
		# print(x)
		# print(costs[x])
		g[x] = sorted(costs[x].keys(), key=lambda g: costs[x][g])
	for x in demand:
	    g[x] = sorted(costs.keys(), key=lambda g: costs[g][x])
	 
	while g:
	    d = {}
	    # print(supply,demand)
	    for x in demand:
	        d[x] = (costs[g[x][1]][x] - costs[g[x][0]][x]) if len(g[x]) > 1 else (costs[g[x][0]][x])
	    s = {}
	    for x in supply:
	        s[x] = (costs[x][g[x][1]] - costs[x][g[x][0]])  if len(g[x]) > 1 else costs[x][g[x][0]]
	    f = max(d, key=lambda n: d[n])
	    t = max(s, key=lambda n: s[n])
	    t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)
	    v = min(supply[f], demand[t])
	    # print(f,t)
	    # print(v)
	    res[f][t] += v
	    demand[t] -= v
	    
	    if demand[t] == 0:
	        for k, n in supply.items():
	            if n != 0:
	                g[k].remove(t)
	        del g[t]
	        del demand[t]
	    supply[f] -= v
	    if supply[f] == 0:
	        for k, n in demand.items():
	            if n != 0:
	                g[k].remove(f)
	        del g[f]
	        del supply[f]
	 
	# print("G",g)
	cost = 0
	# cols = sorted(demand.keys())
	# print(costs)
	for g in sorted(costs1):
	    # print (g, " ",)
	    # print("S")
	    for n in cols:
	        y = res[g][n]
	        # print("YESS",y)
	        if y != 0:
	            pass
	            # print (y,)
	        cost += y * costs1[g][n]
	        # print ("  ",)
	    # print(" ")
	
	costs_list.append(cost)
	

 # 921541
import csv
from itertools import zip_longest
instance_number=range(1,641)
d = [instance_number,costs_list]
export_data = zip_longest(*d, fillvalue = '')
with open('TOM-VOM.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("Instance","Costs"))
      wr.writerows(export_data)
myfile.close()